# Unified LLM Generation Pipeline

This notebook provides an interface to the unified generation pipeline.
All logic lives in `generation.py` - this notebook is for interactive testing and batch runs.

The pipeline handles both Paleolithic and Holocene people automatically based on the sampled birth year.

In [1]:
import dill
from tqdm import tqdm

from generation import (
    generate_person,
    generate_batch,
    generate_batch_parallel,
    # Individual steps if needed
    generate_geography,
    generate_demographics,
    generate_structured_incidents,
    generate_historical_context,
    generate_name,
    generate_narrative,
    quality_check,
    reset_to_stage
)
from llm_utils import GenerationContext, extract_json

from person import sample_person, Person

import copy

# Batch Generation

In [2]:
#sample_person()

Warning 3: Cannot find header.dxf (GDAL_DATA is not defined)


In [3]:
test_people = generate_batch_parallel(n=10, model="gpt-5.2", workers=10)

Sampling 10 people...
Generating 10 people with 10 parallel workers...


100%|███████████████████████████████████████████| 10/10 [03:53<00:00, 23.37s/it]

Done: 10 generated


In [4]:
with open('test_examples.pkl', 'wb') as f:
    dill.dump(test_people,f)

In [5]:
%run export.py test_examples.pkl

Loading people from test_examples.pkl...
Found 10 people
Removing existing markdown files from ../_lives...
  Removed 53 files
  Exported 10 people...

Successfully exported 10 people to ../_lives


## Step-by-Step Generation

For debugging or custom workflows, you can run each step individually.